***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [7]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  PRIMARY_WORKER_COUNT  SECONDARY_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-978e  GCE       4                                             RUNNING  us-central1-a


# Imports & Setup

In [8]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [9]:
import math
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Mar  4 18:34 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [11]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [12]:
spark

In [13]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'irproject-414719bucket' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if b.name != 'graphframes.sh' and not b.name.startswith("postings_gcp") and not b.name.startswith("page_views") and not b.name.startswith("postings_gcp_Title") and not b.name.startswith("bucketText") and not b.name.startswith("bucketTitle") and not b.name.startswith("page_ranks"):
        paths.append(full_path+b.name)

***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [14]:
parquetFile = spark.read.parquet(*paths)
doc_text_pairs = parquetFile.select("anchor_text", "id").rdd

We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [15]:
# Count number of wiki pages
parquetFile.count()

6348910

Let's import the inverted index module. Note that you need to use the staff-provided version called `inverted_index_gcp.py`, which contains helper functions to writing and reading the posting files similar to the Colab version, but with writing done to a Google Cloud Storage bucket.

In [16]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [17]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [18]:
from inverted_index_gcp import InvertedIndex

**YOUR TASK (10 POINTS)**: Use your implementation of `word_count`, `reduce_word_counts`, `calculate_df`, and `partition_postings_and_write` functions from Colab to build an inverted index for all of English Wikipedia in under 2 hours.

A few notes: 
1. The number of corpus stopwords below is a bit bigger than the colab version since we are working on the whole corpus and not just on one file.
2. You need to slightly modify your implementation of  `partition_postings_and_write` because the signature of `InvertedIndex.write_a_posting_list` has changed and now includes an additional argument called `bucket_name` for the target bucket. See the module for more details.
3. You are not allowed to change any of the code not coming from Colab. 

In [19]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ['category', 'references', 'also', 'links', 'extenal', 'see', 'thumb', 'became', 'may']
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)
all_stopwords = english_stopwords.union(corpus_stopwords)
stemmer = PorterStemmer()

NUM_BUCKETS = 124
def token2bucket_id(token):
    return int(_hash(token),16) % NUM_BUCKETS

#####################################################################################################
stemDict = {}

def word_count(ListOfAnchors, id):
    
    sizeOfDoc = 0
    word_counts = Counter()
    
    for _ , text in ListOfAnchors:
        tokens = [token.group() for token in RE_WORD.finditer(text.lower())]

        for token in tokens:
            if (token not in all_stopwords):
                if(token in stemDict):
                    word_counts[stemDict[token]] += 1
                else:
                    stemDict[token] = stemmer.stem(token)
                    word_counts[stemDict[token]] += 1

                sizeOfDoc += 1

    if len(stemDict) >= 20000:
        stemDict.clear()

    return [(token, (id, count, sizeOfDoc)) for token, count in word_counts.items()]

#####################################################################################################

def reduce_word_counts(unsorted_pl):
    return sorted(unsorted_pl, key=lambda x: x[0])

#####################################################################################################

def calculate_df(postings):
    return postings.map(lambda x: (x[0], len(x[1])))

#####################################################################################################

def partition_postings_and_write(postings):

    bucketed_postings = postings.map(lambda x: (token2bucket_id(x[0]), [(x[0], x[1])]))         
    grouped_postings = bucketed_postings.reduceByKey(lambda x, y: x + y)                        
    posting_locations = grouped_postings.map(lambda x: InvertedIndex.write_a_posting_list(x, 'bucketAnchorText', bucket_name))   

    return posting_locations

#####################################################################################################



In [20]:
word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))
word_counts_withNotSize = word_counts.map(lambda x: (x[0], (x[1][0], x[1][1])))

postings = word_counts_withNotSize.groupByKey().mapValues(reduce_word_counts)
postings_filtered = postings.filter(lambda x: len(x[1])>20)

w2df_dict = calculate_df(postings_filtered).collectAsMap()

_ = partition_postings_and_write(postings_filtered).collect()

super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='bucketAnchorText'):
    if not blob.name.endswith("pickle"):
        continue
    with blob.open("rb") as f:
        posting_locs = pickle.load(f)
        for k, v in posting_locs.items():
            super_posting_locs[k].extend(v)




24/03/04 18:52:12 WARN YarnAllocator: Container from a bad node: container_1709577226282_0001_01_000011 on host: cluster-978e-w-2.c.irproject-414719.internal. Exit status: 143. Diagnostics: [2024-03-04 18:52:12.140]Container killed on request. Exit code is 143
[2024-03-04 18:52:12.141]Container exited with a non-zero exit code 143. 
[2024-03-04 18:52:12.142]Killed by external signal
.
24/03/04 18:52:12 WARN YarnSchedulerBackend$YarnSchedulerEndpoint: Requesting driver to remove executor 11 for reason Container from a bad node: container_1709577226282_0001_01_000011 on host: cluster-978e-w-2.c.irproject-414719.internal. Exit status: 143. Diagnostics: [2024-03-04 18:52:12.140]Container killed on request. Exit code is 143
[2024-03-04 18:52:12.141]Container exited with a non-zero exit code 143. 
[2024-03-04 18:52:12.142]Killed by external signal
.
24/03/04 18:52:12 ERROR YarnScheduler: Lost executor 11 on cluster-978e-w-2.c.irproject-414719.internal: Container from a bad node: container_17

In [21]:
Doc_TF = word_counts.map(lambda x: (x[1][0], x[1][2])).distinct()
Doc_TF = Doc_TF.collectAsMap()

24/03/04 19:07:58 WARN YarnAllocator: Container from a bad node: container_1709577226282_0001_01_000022 on host: cluster-978e-w-3.c.irproject-414719.internal. Exit status: 143. Diagnostics: [2024-03-04 19:07:58.146]Container killed on request. Exit code is 143
[2024-03-04 19:07:58.146]Container exited with a non-zero exit code 143. 
[2024-03-04 19:07:58.146]Killed by external signal
.
24/03/04 19:07:58 WARN YarnSchedulerBackend$YarnSchedulerEndpoint: Requesting driver to remove executor 22 for reason Container from a bad node: container_1709577226282_0001_01_000022 on host: cluster-978e-w-3.c.irproject-414719.internal. Exit status: 143. Diagnostics: [2024-03-04 19:07:58.146]Container killed on request. Exit code is 143
[2024-03-04 19:07:58.146]Container exited with a non-zero exit code 143. 
[2024-03-04 19:07:58.146]Killed by external signal
.
24/03/04 19:07:58 ERROR YarnScheduler: Lost executor 22 on cluster-978e-w-3.c.irproject-414719.internal: Container from a bad node: container_17

In [22]:
inverted = InvertedIndex()
inverted.posting_locs = super_posting_locs
inverted.df = w2df_dict
inverted.nf = Doc_TF
inverted.write_index('bucketAnchorText', 'indexAnchorText', bucket_name)

index_src = "indexAnchorText.pkl"
index_dst = f'gs://{bucket_name}/bucketAnchorText/{index_src}'
!gsutil cp $index_src $index_dst

CommandException: No URLs matched: indexAnchorText.pkl


In [23]:
!gsutil ls -lh $index_dst

 55.31 MiB  2024-03-04T19:17:30Z  gs://irproject-414719bucket/bucketAnchorText/indexAnchorText.pkl
TOTAL: 1 objects, 57993678 bytes (55.31 MiB)
